In [ ]:
import os
import json
import torch

from json import JSONDecodeError
import numpy as np
from tqdm import tqdm
from PIL import Image

from transformers import ViTForImageClassification, ViTFeatureExtractor

from collections import defaultdict

from util.constants import Topic, ThumbnailURL, thumbnail_URL
import requests

In [ ]:
# Models

class ImageLatentRepresentationModel(ViTForImageClassification):
    """
    Hook into the ViTForImageClassification Class in order to get the latent
    representations of an image, not just the classification output. Source code
    taken from HuggingFace open-source GitHub:
    https://github.com/huggingface/transformers/blob/main/src/transformers/models/vit/modeling_vit.py
    """

    def __init__(self, config):
        super().__init__(config)

    def forward(self, pixel_values):
        """
        Overwritten forward method to only get latent representation of the image,
        without image classification.

        args:
            - pixel_values: input image, as PyTorch Tensor of shape [1,3,224,224] 
        
        returns:
            - latent_vec: latent vector representing the image
        """
        vit_output = self.vit(pixel_values)
        latent_vec = vit_output[0][:,0,:]

        return latent_vec

def load_model(device, install=True):
    """
    Loads in a pre-trained ViT model for latent image representation

    args:
        - device: what device the model should be on
    returns:
        - model: pre-trained ViT model
        - feature_extractor: pre-trained feature extractor for processing images
    """
    if install:
        print("Installing ViT architecture... This may take a couple of minutes.")
        os.system("pip install -q git+https://github.com/huggingface/transformers.git")
        print("Finished installing.")

    print("Loading pretrained ViT model...")
    model = ImageLatentRepresentationModel.from_pretrained('google/vit-base-patch16-224')
    model.eval()
    model.to(device)
    print("Loaded model")

    feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')

    return model, feature_extractor

def get_latent_vectors(model, ft_extr, raw_imgs, device):
    """
    Function to get the latent representation of an image.

    args:
        - model: The pretrained ViT model
        - ft_extr: The feature extractor, used to preprocess the images
        - raw_imgs: The raw thumbnail images
    """
    # Encode the images using the feature extractor
    encodings = ft_extr(images=raw_imgs, return_tensors="pt")
    pixel_values = encodings['pixel_values'].to(device)

    # Get the latent representation by passing it through the network
    latent_vecs = model(pixel_values)
    del pixel_values
    return latent_vecs.detach().cpu().numpy().tolist()

Get latents for all videos

In [ ]:
# Read data
channel_videos_dict = {}
for cat in Topic._member_names_:
    with open(os.path.join("..", "data", "info_videos", F"videos-info_{cat}.json"), "r") as f:
        channel_videos_dict.update(json.load(f))

In [ ]:
RESULTS_DIR = os.path.join("..", "data", "thumbnail-latents")

video_results_dir = os.path.join("..","..","DATA","thumbnail-latents","videos")
channel_results_dir = os.path.join(RESULTS_DIR, "channels")
categories_results_dir = os.path.join(RESULTS_DIR, "categories")

if not os.path.exists(video_results_dir):
    os.makedirs(video_results_dir)

if not os.path.exists(categories_results_dir):
    os.makedirs(categories_results_dir)

def get_done_list(dir):
    return [nm.replace(".json",'').replace('.pt','') for nm in os.listdir(dir)]

In [ ]:
# Run the code in batches
THUMBNAIL_DIR = os.path.join("..","..","thumbnails")
quality = ThumbnailURL.high

FETCH_THUMBS = False
if not FETCH_THUMBS:
    available_ids = [nm.replace('_high.jpg','') for nm in get_done_list(THUMBNAIL_DIR)]

batch_size = 16

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}\n")

model, feature_extractor = load_model(device, install=False)

for channel,videos in tqdm(channel_videos_dict.items()):
    done_path = os.path.join(RESULTS_DIR, "videos_done", channel+".json")
    ids = [vid["id"] for vid in videos[:30]]
    if not FETCH_THUMBS:
        ids = list(set(ids).intersection(available_ids))
        if not ids:
            continue
    try:
        with open(done_path, "r") as f:
            done_ids = json.load(f)
        if set(ids).issubset(done_ids): # Every id is done
            continue
    except FileNotFoundError:
        pass

    fetched_ids = []
    results = []
    for i in range(2):
        id_batch = ids[i*batch_size:(i+1)*batch_size]

        if FETCH_THUMBS:
            raws = []
            for id in id_batch:
                url = thumbnail_URL(id, quality)
                try:
                    raws.append(requests.get(url, stream=True).raw)
                except:
                    continue
                fetched_ids.append(id)
            images = [np.array(Image.open(raw)) for raw in raws]
        else:
            imgs_paths = [os.path.join(THUMBNAIL_DIR, id+"_high.jpg") for id in id_batch]
            fetched_ids.extend(id_batch)
            images = [Image.open(path) for path in imgs_paths]

        if not images:
            continue

        try:
            results.extend(get_latent_vectors(model, feature_extractor, images, device))
        except Exception as e:
            print(e)
    ids = fetched_ids

    path = os.path.join(video_results_dir, f"{channel}.pt")
    torch.save(np.array(results), path)
    with open(done_path, "w") as f:
        json.dump(ids, f)

    torch.cuda.empty_cache()

Channel stats

In [ ]:
# Read data
channel_videos_dict = {}
for cat in Topic._member_names_:
    with open(os.path.join("..", "data", "info_videos", F"videos-info_{cat}.json"), "r") as f:
        channel_videos_dict.update(json.load(f))

In [ ]:
# Calculate channel results
for channel,videos in tqdm(channel_videos_dict.items()):
    in_path = os.path.join(video_results_dir,channel+".pt")
    try:
        result_list = torch.load(in_path)
    except:
        print(f"couldn't load {channel}")

    channel_mean = result_list.mean(axis=0)
    channel_result = {
        "std": float(result_list.std()),
        "len": len(result_list),
    }

    filepath = os.path.join(channel_results_dir, f"{channel}.json")
    with open(filepath, "w") as f:
        json.dump(channel_result, f)
    filepath = os.path.join(RESULTS_DIR, "channels_mean", f"{channel}.pt")
    torch.save(channel_mean, filepath)

Category stats

In [ ]:
# Read data
with open(os.path.join("..", "data", "channel2category.json"), "r") as f:
    channel2cat = json.load(f)

In [ ]:
# Make list of results per channel for each category
category_results_list = defaultdict(list)
for channel in tqdm(get_done_list(channel_results_dir)):
    cat = channel2cat[channel]
    results = {}
    for folder,ext in [("channels",".json"), ("channels_mean",".pt")]:
        filepath = os.path.join(RESULTS_DIR, folder, channel+ext)
        try:
            if ext == ".json":
                with open(filepath, "r") as f:
                    results.update(json.load(f))
            elif ext == ".pt":
                results.update({"mean": torch.load(filepath)})
        except JSONDecodeError:
            print(f"couldn't open {channel}; deleting file")
            os.remove(filepath)
    category_results_list[cat].append(results)

In [ ]:
# Calculate category results
for cat,stats_list in category_results_list.items():
    mean_list = np.array([channel_stats["mean"] for channel_stats in stats_list if channel_stats["mean"].shape])
    std_list = np.array([channel_stats["std"] for channel_stats in stats_list])
    category_mean = mean_list.mean(axis=0)
    category_result = {
        "std": float(mean_list.std()),
        "avg_std_of_channels": std_list.mean(),
        "len": len(mean_list),
    }

    filepath = os.path.join(RESULTS_DIR, "categories", f"{cat}.json")
    with open(filepath, "w") as f:
        json.dump(category_result, f)
    filepath = os.path.join(RESULTS_DIR, "categories_mean", f"{cat}.pt")
    torch.save(category_mean, filepath)

Most representative

In [ ]:
channels = get_done_list(channel_results_dir)

channel_means = []
for channel in tqdm(channels):
    path = os.path.join(RESULTS_DIR, "channels_mean", channel+".pt")
    channel_mean = torch.load(path)
    if not channel_mean.shape:
        continue
    channel_means.append(channel_mean)
channel_means = torch.Tensor(np.array(channel_means))

In [ ]:
categories = Topic._member_names_

category_means = []
for cat in tqdm(categories):
    path = os.path.join(RESULTS_DIR, "categories_mean", cat+".pt")
    category_means.append(torch.load(path))
category_means = torch.Tensor(category_means)

all_means = torch.cat((channel_means, category_means), axis=0)

In [ ]:
def cos_sim(arrayA, arrayB):
    """[N,D], [M,D] -> [N,M]"""
    normA = torch.norm(arrayA, p=2, dim=1)[:,None]
    normB = torch.norm(arrayB, p=2, dim=1)[:,None]
    return arrayA/normA @ (arrayB/normB).T
    
def dist(tensorA, tensorB):
    """[N,D], [M,D] -> [N,M]"""
    return ((tensorA[:,:,None] - tensorB.transpose()[None,:,:])**2).mean(1)**.5

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

all_means = all_means.to(device)
best_similarity = torch.zeros(len(all_means)).to(device)
best_repr = [""]*len(all_means)

batch_size = 128
for i in tqdm(range(len(channels)//batch_size+1)):
    done_ids = []
    done_means = torch.zeros((0,768), device=device)
    for channel in channels[i*batch_size:(i+1)*batch_size]:
        done_path = os.path.join(RESULTS_DIR, "videos_done", channel+".json")
        try:
            with open(done_path, "r") as f:
                done_ids.extend(json.load(f))
            if not done_ids:
                continue
        except FileNotFoundError:
            print(f"couldn't open {channel}")
            continue
        
        filepath = os.path.join(video_results_dir, channel+".pt")
        loaded_means = torch.Tensor(torch.load(filepath)).to(device)
        done_means = torch.cat((done_means, loaded_means), dim=0)

    if len(done_means) < 1:
        continue

    similarities = torch.max(cos_sim(all_means, done_means), dim=-1)
    update_best = (similarities.values > best_similarity).nonzero()
    best_similarity[update_best] = similarities.values[update_best]
    for idx in update_best:
        best_repr[idx] = done_ids[similarities.indices[idx]]

In [ ]:
channels_best_repr = {k:{"sim": sim.item(), "vid_id": id} for k,sim,id in zip(channels,best_similarity,best_repr) if id}
with open(os.path.join(RESULTS_DIR, "channels_best_repr.json"), "w") as f:
    json.dump(channels_best_repr, f)

In [ ]:
categories_best_repr = {k:{"sim": sim.item(), "vid_id": id} for k,sim,id in zip(categories,best_similarity[-9:],best_repr[-9:]) if id}
with open(os.path.join(RESULTS_DIR, "categories_best_repr.json"), "w") as f:
    json.dump(categories_best_repr, f)

Video latent deviation

In [ ]:
channels = get_done_list(channel_results_dir)

channel2mean = {}
for channel in tqdm(channels):
    path = os.path.join(RESULTS_DIR, "channels_mean", channel+".pt")
    channel2mean[channel] = torch.load(path)

In [ ]:
vid2views = {}
for cat in Topic._member_names_:
    with open(os.path.join("..", "data", "info_videos", F"videos-info_{cat}.json"), "r") as f:
        videos_info = json.load(f)
    vid2views.update({vid["id"]:vid["views"] for channel_vids in tqdm(videos_info.values()) for vid in channel_vids})

In [ ]:
for channel in tqdm(channels):
    done_path = os.path.join(RESULTS_DIR, "videos_done", channel+".json")
    try:
        with open(done_path, "r") as f:
            done_ids = json.load(f)
        if not done_ids:
            continue
    except FileNotFoundError:
        print(f"couldn't open {channel}")
        continue
    
    filepath = os.path.join(video_results_dir, channel+".pt")
    loaded_means = torch.load(filepath)
    done_means = loaded_means

    if len(done_means) < 1:
        continue

    distances = dist(channel2mean[channel][None], done_means)[0]

    result = {
        "mean": float(distances.mean()),
        "datapoints": [{
            "x": float(d),
            "y": vid2views[id],
            "id": id,
        } for d,id in zip(distances, done_ids)]
    }

    with open(os.path.join(RESULTS_DIR, "video_deviation", channel+".json"), "w") as f:
        json.dump(result, f)